In [1]:
import numpy as np
import pandas as pd
import seaborn as sns 

In [2]:
df = sns.load_dataset('titanic')
df.head(3)

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True


* 데이터 전처리

In [3]:
# Feature selection
df = df[['survived','pclass','sex','age','sibsp','parch','fare','embarked','deck']]
df.head(3)

,survived,pclass,sex,age,sibsp,parch,fare,embarked,deck
0,0,3,male,22.0,1,0,7.2500,S,NaN
1,1,1,female,38.0,1,0,71.2833,C,C
2,1,3,female,26.0,0,0,7.9250,S,NaN


In [4]:
# NaN 처리
df.isna().sum()

survived      0
pclass        0
sex           0
age         177
sibsp         0
parch         0
fare          0
embarked      2
deck        688
dtype: int64

In [5]:
# age컬럼은 평균으로 대체
df.age.fillna(df.age.mean(), inplace=True)
df.age.isna().sum()

0

In [8]:
df.embarked.value_counts()

S    646
C    168
Q     77
Name: embarked, dtype: int64

In [9]:
# embarked는 최빈값으로 대체
df.embarked.fillna('S', inplace=True)
df.embarked.isna().sum()

0

In [10]:
# deck는 삭제
df.drop(columns=['deck'], inplace=True)
df.head(3)

,survived,pclass,sex,age,sibsp,parch,fare,embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S


In [12]:
# 카테고리 값인 sex, embarked를 숫자로 변환
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['sex'] = le.fit_transform(df.sex)
df['embarked'] = le.fit_transform(df.embarked)
df.head(3)

,survived,pclass,sex,age,sibsp,parch,fare,embarked
0,0,3,1,22.0,1,0,7.2500,2
1,1,1,0,38.0,1,0,71.2833,0
2,1,3,0,26.0,0,0,7.9250,2


### 2. Train / Test dataset으로 분리

In [15]:
# X와 y를 numpy 배열로
X = df.iloc[:,1:].values 
y = df.survived.values
X.shape, y.shape

((891, 7), (891,))

In [16]:
df.survived.value_counts()

0    549
1    342
Name: survived, dtype: int64

In [17]:
from sklearn.model_selection import train_test_split

In [18]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, test_size=0.2, random_state=2021
)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((712, 7), (179, 7), (712,), (179,))

In [19]:
np.unique(y_train, return_counts=True)

(array([0, 1], dtype=int64), array([439, 273], dtype=int64))

In [20]:
print(342*439, 549*273)

150138 149877


### 3. RandomForest 모델 생성 및 학습

In [11]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(random_state=2021)
rfc.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 2021,
 'verbose': 0,
 'warm_start': False}

In [21]:

rfc.fit(X_train, y_train)

RandomForestClassifier(random_state=2021)

### 4. 모델 예측 및 평가

In [34]:

rfc.score(X_test, y_test)

0.8100558659217877

### 5. (3, 4) 대신에 GridSearchCV 사용

In [36]:
# GridSearchCV 수행
params = {
    'max_depth': [2, 4, 6, 8],
    'min_samples_split': [2,4,6]
}

In [37]:
from sklearn.model_selection import GridSearchCV

grid_rf = GridSearchCV(rfc, param_grid=params, scoring='accuracy', cv=5)
grid_rf.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=2021),
             param_grid={'max_depth': [2, 4, 6, 8],
                         'min_samples_split': [2, 4, 6]},
             scoring='accuracy')

In [38]:
grid_rf.best_params_

{'max_depth': 8, 'min_samples_split': 4}

In [39]:
params = {
    'max_depth': [7,8,9,10],
    'min_samples_split': [3,4,5]
}

In [40]:
grid_rf = GridSearchCV(rfc, param_grid=params, scoring='accuracy', cv=3)
grid_rf.fit(X_train, y_train)

GridSearchCV(cv=3, estimator=RandomForestClassifier(random_state=2021),
             param_grid={'max_depth': [7, 8, 9, 10],
                         'min_samples_split': [3, 4, 5]},
             scoring='accuracy')

In [42]:
grid_rf.best_params_

{'max_depth': 7, 'min_samples_split': 5}

In [43]:
grid_rf.score(X_test, y_test)

0.8268156424581006

### 6. 테스트 데이터에 적용

In [52]:
# 테스트 데이터에 적용
X_test[12], y_test[12]

(array([ 2. ,  1. , 16. ,  0. ,  0. , 10.5,  2. ]), 0)

In [53]:
grid_rf.predict(X_test[100].reshape(1,-1))

array([0], dtype=int64)

* 타이타닉 엉터리 분류기

In [54]:
from sklearn.base import BaseEstimator

class MyClassfier(BaseEstimator):
    # fit(), predict() method는 재정의(overriding)
    def fit(self, X, y):
        pass
    def predict(self, X):
        pred = np.zeros(X.shape[0])
        for i in range(X.shape[0]):
            if X[i,1] == 0:             # 여성이면 생존으로 처리
                pred[i] = 1
        return pred

In [61]:
my_clf = MyClassfier()
my_clf.fit(X_train, y_train)
pred_my = my_clf.predict(X_test)

In [62]:
X_test[:5, 1], pred_my[:5]

(array([0., 1., 0., 1., 0.]), array([1., 0., 1., 0., 1.]))

In [63]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, pred_my)

0.7877094972067039

In [64]:
pred_rf = best_rf.predict(X_test)
accuracy_score(y_test, pred_rf)

NameError: name 'best_rf' is not defined

In [65]:
sdf = pd.DataFrame({'y_test':y_test, 'RF':pred_rf, 'My':pred_my})
sdf.head()

NameError: name 'pred_rf' is not defined

- 모델의 성능을 평가할 때 무조건적으로 정확도를 사용하는 것은 지양

In [67]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score
confusion_matrix(y_test, pred_rf)

NameError: name 'pred_rf' is not defined

In [68]:
confusion_matrix(y_test, pred_my)

array([[95, 15],
       [23, 46]], dtype=int64)

In [69]:
precision_score(y_test, pred_rf), recall_score(y_test, pred_rf)

NameError: name 'pred_rf' is not defined

In [70]:
 precision_score(y_test, pred_my), recall_score(y_test, pred_my)

(0.7540983606557377, 0.6666666666666666)